# LangGraph から MCP ツールを呼び出すシンプルな例

- https://github.com/langchain-ai/langchain-mcp-adapters
- https://www.youtube.com/watch?v=OX89LkTvNKQ


In [3]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

## 単純な計算ツールを使う例


In [17]:
# # math_server.py
# from mcp.server.fastmcp import FastMCP

# mcp = FastMCP("Math")


# @mcp.tool()
# def add(a: int, b: int) -> int:
#     """Add two numbers"""
#     return a + b


# @mcp.tool()
# def multiply(a: int, b: int) -> int:
#     """Multiply two numbers"""
#     return a * b


# if __name__ == "__main__":
#     mcp.run(transport="stdio")

In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

# MCPサーバのコマンドと引数を指定
server_params = StdioServerParameters(
    command="python", args=["/home/mori/learning-langchain/langgraph/math_server.py"]
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)

        # LangGraphのエージェントを作成
        agent = create_react_agent(model, tools)
        response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})
        for message in response["messages"]:
            message.pretty_print()

================================ Human Message =================================

what's (3 + 5) x 12?
================================== Ai Message ==================================
Tool Calls:
  add (call_q6UvrENLwmaENojLnyTauDZa)
 Call ID: call_q6UvrENLwmaENojLnyTauDZa
  Args:
    a: 3
    b: 5
  multiply (call_rNlT6OOjVXUYC2hzUHRuSh74)
 Call ID: call_rNlT6OOjVXUYC2hzUHRuSh74
  Args:
    a: 12
    b: 1
================================= Tool Message =================================
Name: add

8
================================= Tool Message =================================
Name: multiply

12
================================== Ai Message ==================================
Tool Calls:
  multiply (call_cHMxWLYxmQwguvbthnKNCZBh)
 Call ID: call_cHMxWLYxmQwguvbthnKNCZBh
  Args:
    a: 8
    b: 12
================================= Tool Message =================================
Name: multiply

96
================================== Ai Message ==================================

The result 

## 複数の MCP サーバのツールを使う例


In [19]:
# from typing import List

# from mcp.server.fastmcp import FastMCP

# mcp = FastMCP("Weather")


# @mcp.tool()
# async def get_weather(location: str) -> str:
#     """Get weather for location."""
#     return "It's always sunny in " + location


# if __name__ == "__main__":
#     mcp.run(transport="sse")

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

async with MultiServerMCPClient(
    {
        "math": {
            "command": "python",
            "args": ["/home/mori/learning-langchain/langgraph/math_server.py"],
            "transport": "stdio",
        },
        "weather": {
            # SSE (Server-Sent Events) 経由での接続
            # こちらは事前にスクリプトを起動しておく必要がある
            # python weather_server.py
            "url": "http://localhost:8000/sse",
            "transport": "sse",
        },
    }
) as client:
    agent = create_react_agent(model, client.get_tools())

    # 計算ツール呼び出し
    response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})
    for message in response["messages"]:
        message.pretty_print()

    # 天気ツール呼び出し
    response = await agent.ainvoke({"messages": "What's the weather in Tokyo?"})
    for message in response["messages"]:
        message.pretty_print()

[04/24/25 15:03:05] INFO     Connecting to SSE endpoint: http://localhost:8000/sse                        ]8;id=995729;file:///home/mori/learning-langchain/.venv/lib/python3.12/site-packages/mcp/client/sse.py\sse.py]8;;\:]8;id=775587;file:///home/mori/learning-langchain/.venv/lib/python3.12/site-packages/mcp/client/sse.py#45\45]8;;\

                    INFO     HTTP Request: GET http://localhost:8000/sse "HTTP/1.1 200 OK"          ]8;id=604763;file:///home/mori/learning-langchain/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=494679;file:///home/mori/learning-langchain/.venv/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received endpoint URL:                                                       ]8;id=545454;file:///home/mori/learning-langchain/.venv/lib/python3.12/site-packages/mcp/client/sse.py\sse.py]8;;\:]8;id=245147;file:///home/mori/learning-langchain/.venv/lib/python3.12/site-packages/mcp/client/sse.py#65\65]8;;\
                             http://localhost:8000/messages/?session_id=4c99c97e0a5a4aaeb62ab3b946a195a5           

                    INFO     Starting post writer with endpoint URL:                                     ]8;id=288666;file:///home/mori/learning-langchain/.venv/lib/python3.12/site-packages/mcp/client/sse.py\sse.py]8;;\:]8;id=633903;file:///home/mori/learning-langchain/.venv/lib/python3.12/site-packages/mcp/client/sse.py#135\135]8;;\
                             http://localhost:8000/messages/?session_id=4c99c97e0a5a4aaeb62ab3b946a195a5           

                    INFO     HTTP Request: POST                                                     ]8;id=480516;file:///home/mori/learning-langchain/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=505654;file:///home/mori/learning-langchain/.venv/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             http://localhost:8000/messages/?session_id=4c99c97e0a5a4aaeb62ab3b946a                
                             195a5 "HTTP/1.1 202 Accepted"                                                         

...

================================ Human Message =================================

what's (3 + 5) x 12?
================================== Ai Message ==================================
Tool Calls:
  add (call_SY7PClYi8PiFVOX1BRtF9Oba)
 Call ID: call_SY7PClYi8PiFVOX1BRtF9Oba
  Args:
    a: 3
    b: 5
  multiply (call_mEAdLQ6ErbzWlcDipFvtgUfd)
 Call ID: call_mEAdLQ6ErbzWlcDipFvtgUfd
  Args:
    a: 12
    b: 1
================================= Tool Message =================================
Name: add

8
================================= Tool Message =================================
Name: multiply

12
================================== Ai Message ==================================
Tool Calls:
  multiply (call_k06xjDgacL4tK9KsNTEMZ4qa)
 Call ID: call_k06xjDgacL4tK9KsNTEMZ4qa
  Args:
    a: 8
    b: 12
================================= Tool Message =================================
Name: multiply

96
================================== Ai Message ==================================

The result 

                    INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=588934;file:///home/mori/learning-langchain/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=932467;file:///home/mori/learning-langchain/.venv/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             "HTTP/1.1 200 OK"                                                                     

                    INFO     HTTP Request: POST                                                     ]8;id=398106;file:///home/mori/learning-langchain/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=559935;file:///home/mori/learning-langchain/.venv/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             http://localhost:8000/messages/?session_id=4c99c97e0a5a4aaeb62ab3b946a                
                             195a5 "HTTP/1.1 202 Accepted"                                                         

[04/24/25 15:03:10] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=198810;file:///home/mori/learning-langchain/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=16727;file:///home/mori/learning-langchain/.venv/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             "HTTP/1.1 200 OK"                                                                     

================================ Human Message =================================

What's the weather in Tokyo?
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_stwoOot9nEzQhNsRpd4uAI0n)
 Call ID: call_stwoOot9nEzQhNsRpd4uAI0n
  Args:
    location: Tokyo
================================= Tool Message =================================
Name: get_weather

It's always sunny in Tokyo
================================== Ai Message ==================================

The weather in Tokyo is clear and sunny.
